In [2]:
pip install names

  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803688 sha256=69433bb1d72ca3ec665f660115ed03a25b16e850bf4710b071c6ca34d4f36e94
  Stored in directory: c:\users\djhuber\appdata\local\pip\cache\wheels\d0\35\f7\c72132a4f3878b82018a3e61bf2a35e6b63cebe1dd9f72ec1e
Successfully built names
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sqlite3 as sq
import pandas as pd
import random as rd 
import matplotlib.pyplot as plt
import names as nm
from IPython.display import clear_output # This is so we can clear output and make the UI easier to look at
# importing all the modules we need ya know how it is
# below is reading our data and displaying a pandas dataframe
elazteca_df = pd.read_csv('DATA.csv')
display(elazteca_df.head())

,ItemID,Name,Typeofdish,Price
0,0,Tortilla Soup,Soups & Salads,8.75
1,1,Taco Salad,Soups & Salads,9.25
2,2,Guacamole Salad,Soups & Salads,4.25
3,3,Tossed Salad,Soups & Salads,3.75
4,4,Ceasar Salad,Soups & Salads,9.75


In [6]:
# writing our first table into our database
# Below is the code for writing the Inventory table
with open('database.sql', mode='w') as i_table:
    i_table.write('DROP TABLE IF EXISTS Inventory;\n')
    i_table.write('CREATE TABLE Inventory (ItemID REAL, Name TEXT, Typeofdish TEXT, Price REAL);\n')
    i_table.write('INSERT INTO Inventory (ItemID, Name, Typeofdish, Price) VALUES\n')
    for i in range(elazteca_df.shape[0]):
        i_table.write('('+str(elazteca_df.iloc[i]['ItemID'])+',') 
        i_table.write('\''+str(elazteca_df.iloc[i]['Name'])+'\',')
        i_table.write('\''+str(elazteca_df.iloc[i]['Typeofdish'])+'\',')
        i_table.write(str(elazteca_df.iloc[i]['Price'])+')')
        if i < elazteca_df.shape[0]-1:
            i_table.write(',\n')
        else:
            i_table.write(';\n')

In [7]:
# Writing our Transaction table into our database
# The variable 'rando' is a dataframe I made to randomly select orderitems for making transaction data. 
# I combined the Item ID, Name and, Price columns of our inventory table then random selected an ItemID -
# - then used that ItemID to get the name and price of that particular menu item
rando = pd.DataFrame().assign(ID = elazteca_df['ItemID'], Name = elazteca_df['Name'], Price = elazteca_df['Price'])

# I know the code below is messy but it works well.
with open('database.sql', mode='a') as t_table:
    t_table.write('DROP TABLE IF EXISTS Transactions;\n')
    t_table.write('CREATE TABLE Transactions (TransactionID REAL, CustomerID REAL, OrderItems TEXT, Price TEXT, TimeStamp TEXT, CardNumber TEXT);\n')
    t_table.write('INSERT INTO Transactions (TransactionID, CustomerID, OrderItems, Price, TimeStamp, CardNumber) VALUES\n')                
    for i in range(200): # 20 Transactions are being create 
        random = rd.choice(rando['ID']) # randomly selecting ItemID
        hour = rd.randint(11, 20)
        minute = rd.randint(1, 60)
        if minute < 10: # Adding a 0 if the minute is below 10
            minute = '0' + str(minute)
        time = (str(hour) + ':' + str(minute))
        month = rd.randint(1,12)
        day = rd.randint(1,31)
        year = rd.randint(2020,2022)
        date = (str(month)+ '/' + str(day)+ '/' +str(year))
        t_table.write('('+str(i)+',') # Transaction ID
        t_table.write(str(rd.randint(1, 21))+',') # Random Customer ID
        t_table.write('\''+str(rando.iloc[random]['Name'])+'\',') # Getting the Name with that random ItemID choice
        t_table.write('\''+str(rando.iloc[random]['Price'])+'\',') # Getting the Price with that random ItemID choice
        t_table.write('\''+str(time)+' '+str(date)+'\',') # TimeStamp
        t_table.write(str(rd.randrange(1000,9999))+')') # Random CardNumber
        if i < 199:
            t_table.write(',\n')
        else:
            t_table.write(';\n')

In [8]:
# Writing our Customer table
with open('database.sql', mode='a') as c_table:
    c_table.write('DROP TABLE IF EXISTS Customer;\n')
    c_table.write('CREATE TABLE Customer (CustomerID REAL, Name TEXT);\n')
    c_table.write('INSERT INTO Customer (CustomerID, Name) VALUES\n')                
    for i in range(20): # 500 Customers are being created
        c_table.write('('+str(i)+',') # CustomerID
        c_table.write('\''+str(nm.get_full_name())+'\')') # Random full Name
        if i < 19:
            c_table.write(',\n')
        else:
            c_table.write(';\n')

In [9]:
# Changing our databse from sql to db
!sqlite3 database.db < ./database.sql

In [10]:
# Creating a connection to our database and testing a query
conn = sq.connect('database.db')
curs = conn.cursor()
for row in curs.execute("""SELECT * FROM Transactions WHERE CustomerID = 2"""):
    print(row)

(1.0, 2.0, 'Milk', '2.5', '15:35 5/3/2021', '6770')
(3.0, 2.0, '#3CD', '10.45', '15:48 9/13/2021', '3651')
(23.0, 2.0, 'Chimichanga Lunch', '7.45', '13:34 9/18/2021', '5783')
(26.0, 2.0, 'Beef Steak Jalisco', '13.45', '14:39 8/13/2022', '5157')
(36.0, 2.0, 'One taco rice and beans', '4.25', '20:27 3/27/2022', '8096')
(49.0, 2.0, 'Fried Ice Cream', '5.75', '13:60 5/20/2022', '9173')
(62.0, 2.0, 'Chimichanga Lunch', '7.45', '11:17 9/17/2020', '9148')
(89.0, 2.0, 'Lunch Special #7', '7.45', '13:11 1/2/2020', '7535')
(90.0, 2.0, 'Camarones Rancheros', '13.95', '18:35 7/10/2020', '9296')
(96.0, 2.0, 'Camarones a la Diabla', '12.95', '15:06 1/15/2020', '2264')
(114.0, 2.0, 'Taco Salad', '9.25', '14:20 7/8/2022', '8198')
(132.0, 2.0, '12 oz Small', '2.5', '19:42 8/10/2021', '9538')
(137.0, 2.0, 'Mello Yello', '2.99', '13:05 10/28/2020', '6326')
(140.0, 2.0, 'Shredded Cheese', '1.75', '12:04 5/12/2021', '3443')
(165.0, 2.0, 'Cheese', '3.0', '17:02 12/25/2021', '4127')


In [ ]:
# Fully built UI for indivudals who can program SQL and for individuals who cannot program SQL
print('▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇')
print('█    El Azteca Database     █')
print('▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇')
continueTeca = True
while (continueTeca): # Below is the Main Menu
    print('\n')
    print('Welcome: Main Menu')
    print(" ")
    print('Q   | Logout')
    print('E   | Enter a custom SQL query')
    print('P   | Display a table')
    print(" ")
    print('-Inventory Table Menu-')
    print('R   | Retrieve information of a menu item')
    print('A   | Add a menu item')
    print('D   | Remove a menu item')
    print('U   | Update a menu item')
    print(" ")
    print('-Transaction Table Menu-')
    print('AT  | Add a transaction record') 
    print('DT  | Remove a transaction record')
    print('UT  | Update a transaction record')
    print(" ")
    print('-Customer Table Menu-')
    print('AC  | Add a customer record')
    print('DC  | Remove a customer record')
    print('UC  | Update a customer record')
    print(" ")
    print('-Visual Grpah-')
    print('G   | Top 10 highest repeating customers')
    print('O   | Top 10 most bought items')
    user_choice = input('Select menu: ')
    
# INVENTORY TABLE BELOW ---------------------------------------------------------------------------------------------------
    if user_choice.upper() == 'Q': # Logout
        continueTeca = False
        clear_output()
        
    elif user_choice.upper() == 'E': # Enter a custom SQL query
        print(" ")
        query = input('Please enter your query: ')
        clear_output()
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(pd.read_sql(query, conn))
        
    elif user_choice.upper() == 'P': # Display an entire database
        print(" ")
        table = input('Enter the table name you would like displayed: ')
        clear_output()
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(pd.read_sql("""SELECT * FROM '"""+ table +"""'""", conn))
        
    elif user_choice.upper() == 'R': # Retrieve information of a menu item
        print(" ")
        name = input('Please enter the name of the menu item which you would like information on: ')
        clear_output()
        item = pd.read_sql("""SELECT * FROM Inventory WHERE Name = '"""+ name +"""'""", conn)
        count = pd.read_sql("""SELECT COUNT(*) FROM Transactions WHERE OrderItems = '"""+ name +"""'""", conn)
        print(f'{item.to_string(index=False)}')
        print(" ")
        print(f'{count.to_string(index=False)} - Amount Sold to Date')
        
    elif user_choice.upper() == 'A': # Add a menu item
        print(" ")
        item_id = input('Enter the item ID of the new menu item: ')
        name = input('Enter the name of the new item: ')
        type_of_dish = input('Enter the type of dish: ')
        price = input('Enter the new menu price: ')
        curs.execute("""INSERT INTO Inventory(ItemID, Name, Typeofdish, Price) VALUES (?, ?, ?, ?)""", (item_id, name, type_of_dish, price))
        conn.commit()
        print(f'-{name}- has been added to the inventory table.')
    
    elif user_choice.upper() == 'D': # Remove a menu item
        print(" ")
        name = input('Please enter the item name which you would like to remove: ')
        curs.execute("""DELETE FROM Inventory WHERE Name = '"""+ name +"""'""")
        conn.commit()
        print(f'-{name}- has been deleted from the inventory table.')

    elif user_choice.upper() == 'U': # Update a menu item / Inventory Table Menu
        print('\n')
        print('-Inventory Table Menu-')
        print('Q  | Return to main menu')
        print('N  | Update the name of a menu item')
        print('P  | Update the price of a menu item')
        print('T  | Update the type of dish of a menu item')
        print(" ")
        user_choice = input('Select menu: ')
        
        if user_choice.upper() == 'Q': # Return to main menu
            print(" ")
            
        elif user_choice.upper() == 'N': # Update the name of a menu item
            print(" ")
            name = input('Please enter the name of the menu item you would like to update: ')
            updated_name = input('Enter the new name: ')
            curs.execute("""UPDATE Inventory SET Name = '"""+ updated_name +"""' WHERE Name = '"""+ name + """'""")
            conn.commit()
            clear_output()
            print(f'The name for -{name}- has been updated')
            
        elif user_choice.upper() == 'P': # Update the price of a menu item
            print(" ")
            name = input('Please enter the name of the menu item you would like to update: ')
            updated_price = input('Enter the new price: ')
            curs.execute("""UPDATE Inventory SET Price = '"""+ updated_price +"""' WHERE Name = '"""+ name + """'""")
            conn.commit()
            clear_output()
            print(f'The price for -{name}- has been updated')
            
        elif user_choice.upper() == 'T': # Update the type of dish of a menu item
            print(" ")
            name = input('Please enter the name of the menu item you would like to update: ')
            updated_tod = input('Enter the new type of dish: ')
            curs.execute("""UPDATE Inventory SET Typeofdish = '"""+ updated_tod +"""' WHERE Name = '"""+ name + """'""")
            conn.commit()
            clear_output()
            print(f'The type of dish for -{name}- has been updated')
            
        else: # This displays if the user input does not match any of the inventory table menu choices
            print('Invalid menu choice. Enter again.')
        
# TRANSACTION TABLE BELOW -------------------------------------------------------------------------------------------------       
    elif user_choice.upper() == 'AT': # Add a transaction record
        print(" ")
        trans_id = input('Enter the new transaction ID: ')
        order = input('Enter the name of the menu item purchased: ')
        price = input('Enter the items price: ')
        time = input('Enter the time of the transaction (example- 11:30): ')
        card = input('Enter the last four digits of the customers card number: ')
        curs.execute("""INSERT INTO Transaction(TransactionID, OrderItems, Price, TimeStamp, CardNumber) VALUES (?, ?, ?, ?, ?)""", (trans_id, order, price, time, card))
        conn.commit()
        clear_output()
        print(f'-{name}- has been added to the inventory table.')
        
    elif user_choice.upper() == 'DT': # Delete a transaction record
        print(" ")
        name = input('Please enter the transaction ID for which transaction you would like to remove: ')
        curs.execute("""DELETE * FROM Transactions WHERE TransactionID = '"""+ name +"""'""")
        conn.commit()
        clear_output()
        print(f'-{name}- has been deleted from the transaction table.')
        
    elif user_choice.upper() == 'UT': # Update a transaction table record / Transaction Table Menu
        print('\n')
        print('-Transaction Table Menu-')
        print('Q  | Return to main menu')
        print('I  | Update the transaction ID of an entry')
        print('O  | Update the order items of an entry')
        print('P  | Update the price of an entry')
        print('T  | Update the time of an entry')
        print('C  | Update the card number of an entry')
        print(" ")
        
        user_choice = input('Select menu: ')
        
        if user_choice.upper() == 'Q': # Return to main menu (transaction table)
            print(" ")
            
        elif user_choice.upper() == 'I': # Update the transaction ID of an entry (transaction table)
            print(" ")
            t_id = input('Please enter the transaction ID of the transaction you would like to update: ')
            updated_id = input('Enter the transaction ID: ')
            curs.execute("""UPDATE Transactions SET Name = '"""+ updated_id +"""' WHERE TransactionID = '"""+ t_id + """'""")
            conn.commit()
            clear_output()
            print(f'The transaction ID for transaction -{t_id}- has been updated')
            
        elif user_choice.upper() == 'O': # Update the order items of an entry (transaction table)
            print(" ")
            t_id = input('Please enter the transaction ID of the transaction you would like to update: ')
            updated_items = input('Enter the new order item: ')
            curs.execute("""UPDATE Transactions SET Price = '"""+ updated_items +"""' WHERE TransactionID = '"""+ t_id + """'""")
            conn.commit()
            clear_output()
            print(f'The order items for transaction -{t_id}- has been updated')
            
        elif user_choice.upper() == 'P': # Update the price of an entry (transaction table)
            print(" ")
            t_id = input('Please enter the transaction ID of the transaction you would like to update: ')
            updated_price = input('Enter the new price: ')
            curs.execute("""UPDATE Transactions SET Typeofdish = '"""+ updated_price +"""' WHERE Name = '"""+ t_id + """'""")
            conn.commit()
            clear_output()
            print(f'The price for transaction -{t_id}- has been updated')
            
        elif user_choice.upper() == 'T': # Update the time of an entry (transaction table)
            print(" ")
            t_id = input('Please enter the transaction ID of the transaction you would like to update: ')
            updated_time = input('Enter the new time stamp and date (ex: 11:30 4/20/2020): ')
            curs.execute("""UPDATE Transactions SET Typeofdish = '"""+ updated_time +"""' WHERE Name = '"""+ t_id + """'""")
            conn.commit()
            clear_output()
            print(f'The time stamp for transaction -{t_id}- has been updated')
            
        elif user_choice.upper() == 'C': # Update the card number of an entry (transaction table)
            print(" ")
            t_id = input('Please enter the transaction ID of the transaction you would like to update: ')
            updated_card = input('Enter the new card number: ')
            curs.execute("""UPDATE Transactions SET Typeofdish = '"""+ updated_card +"""' WHERE Name = '"""+ t_id + """'""")
            conn.commit()
            clear_output()
            print(f'The card number for transaction -{t_id}- has been updated')
            
        else: # This displays if the user input does not match any of the transaction table menu choices
            print('Invalid menu choice. Enter again.')
        
# CUSTOMER TABLE BELOW ---------------------------------------------------------------------------------------------------
    elif user_choice.upper() == 'AC': # Add a customer record
        print(" ")
        cust_id = input('Enter the new customer ID: ')
        name = input('Enter the new name of the customer: ')
        curs.execute("""INSERT INTO Customer (CustomerID, Name) VALUES (?, ?)""", (cust_id, name))
        conn.commit()
        clear_output()
        print(f'-{name}- has been added to the Customer table.')
        
    elif user_choice.upper() == 'DC': # Delete a customer record
        print(" ")
        name = input('Please enter the name of the customer you would like to remove: ')
        curs.execute("""DELETE FROM Customer WHERE Name = '"""+ name +"""'""")
        conn.commit()
        clear_output()
        print(f'-{name}- has been deleted from the Customer table.') 
        
    elif user_choice.upper() == 'UC': # Update a customer table record / Customer Table Menu
        print('\n')
        print('-Customer Table Menu-')
        print('Q  | Return to main menu')
        print('N  | Update the customer ID of an entry')
        print('P  | Update the customer name of an entry')
        print(" ")
        user_choice = input('Select menu: ')
        
        if user_choice.upper() == 'Q': # Retrun to main menu (Customer table)
            print(" ")
            
        elif user_choice.upper() == 'N': # Update the customer ID of an entry (Customer table)
            print(" ")
            c_id = input('Please enter the customer ID you would liked updated: ')
            updated_name = input('Enter the new ID: ')
            curs.execute("""UPDATE Customer SET CustomerID = '"""+ updated_name +"""' WHERE CustomerID = '"""+ c_id + """'""")
            conn.commit()
            clear_output()
            print(f'The customer ID for customer -{c_id}- has been updated')
            
        elif user_choice.upper() == 'P': # Update the customer name of an entry (Customer table)
            print(" ")
            c_id = input('Please enter the customer ID you would liked updated: ')
            updated_name = input('Enter the new customer name: ')
            curs.execute("""UPDATE Customer SET Name = '"""+ updated_name +"""' WHERE CustomerID = '"""+ c_id + """'""")
            conn.commit()
            clear_output()
            print(f'The name for customer -{c_id}- has been updated')
            
        else: # This displays if the user input does not match any of the customer table menu choices
            print('Invalid menu choice. Enter again.')

# VISUAL GRAPHS BELOW ----------------------------------------------------------------------------------------------------
    elif user_choice.upper() == 'G': # Top 10 Customers
        clear_output()
        top_10 = pd.read_sql("""SELECT CustomerID,
                        COUNT(CustomerID)
                        FROM Transactions 
                        GROUP BY CustomerID 
                        ORDER BY Count(CustomerID) DESC
                        LIMIT 10""", conn)
        
        top_10.plot(x ='CustomerID', y='COUNT(CustomerID)', kind='bar')
        plt.show()
        
    elif user_choice.upper() == 'O': # Top 10 most bought items
        clear_output()
        top_10 = pd.read_sql("""SELECT OrderItems,
                        COUNT(OrderItems)
                        FROM Transactions 
                        GROUP BY OrderItems 
                        ORDER BY Count(OrderItems) DESC
                        LIMIT 10""", conn)

    
        top_10.plot(x ='OrderItems', y='COUNT(OrderItems)', kind='bar')
        plt.show()
        
    else: # This displays if the user input does not match any of the menu choices
        print('Invalid menu choice. Enter again.')

,CustomerID,Name
0,0.0,Reanna Smith
1,1.0,Melvina Moore
2,2.0,Jennie Dieteman
3,3.0,Carol Troxler
4,4.0,Tammy Anderson
5,5.0,Robert Jones
6,6.0,Eric Mason
7,7.0,Cynthia Naval
8,8.0,Harold Dayley
9,9.0,Lillian Venezia




Welcome: Main Menu
 
Q   | Logout
E   | Enter a custom SQL query
P   | Display a table
 
-Inventory Table Menu-
R   | Retrieve information of a menu item
A   | Add a menu item
D   | Remove a menu item
U   | Update a menu item
 
-Transaction Table Menu-
AT  | Add a transaction record
DT  | Remove a transaction record
UT  | Update a transaction record
 
-Customer Table Menu-
AC  | Add a customer record
DC  | Remove a customer record
UC  | Update a customer record
 
-Visual Grpah-
G   | Top 10 highest repeating customers
O   | Top 10 most bought items
